In [ ]:
import json


# === Function to delete all existing data (optional cleanup) ===
def delete_all_data(tx):
    tx.run("MATCH (n) DETACH DELETE n")
    print("🗑️ Cleared existing data.")

# === Function to insert an entity ===
def insert_entity(tx, name, entity_type, singular, description):
    query = (
        "MERGE (e:Entity {name: $name}) "
        "SET e.entity_type = $entity_type, e.singular = $singular, e.description = $description"
    )
    tx.run(query, name=name, entity_type=entity_type, singular=singular, description=description)
    print(f"✅ Inserted Entity: {name}")

# === Function to insert a relationship ===
def insert_relationship(tx, entity_1, entity_2, rel_type, description):
    query = (
        "MATCH (a:Entity {name: $entity_1}), (b:Entity {name: $entity_2}) "
        "MERGE (a)-[r:RELATION {type: $rel_type, description: $description}]->(b)"
    )
    tx.run(query, entity_1=entity_1, entity_2=entity_2, rel_type=rel_type, description=description)
    print(f"🔗 Created Relationship: ({entity_1})-[:{rel_type}]->({entity_2})")

# === Load JSON data ===
with open("/Users/shiva/Documents/Regal/data/enities_realtions.json", "r") as f:
    kg_data = json.load(f)

# === Insert all entities and relationships into Neo4j ===
with driver.session() as session:
    # Optional: Wipe database before inserting new data
    session.write_transaction(delete_all_data)

    # Insert entities
    for entity in kg_data["entities"]:
        session.write_transaction(
            insert_entity,
            entity["name"],
            entity["entity_type"],
            entity["singular"],
            entity["description"]
        )

    # Insert relationships
    for rel in kg_data["relationships"]:
        session.write_transaction(
            insert_relationship,
            rel["entity_1"],
            rel["entity_2"],
            rel["relationship_type"],
            rel["description"]
        )

print("✅ All entities and relationships inserted into Neo4j.")

# === Close connection ===
driver.close()